# Semi-Supervisée

In [1]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# I. Découpage de la base en apprentissage/test

In [70]:
data = pd.read_csv('./Wave.txt', delimiter=' ', header=None)
y = data[40]
data = data.drop(columns=[40])
print('labels')
print(y)
data

labels
0       2
1       0
2       1
3       0
4       1
       ..
4995    0
4996    1
4997    2
4998    0
4999    0
Name: 40, Length: 5000, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,-0.23,-1.21,1.20,1.23,-0.10,0.12,2.49,1.19,1.34,0.58,...,-0.63,-0.86,-0.70,0.51,0.34,-0.13,-0.87,0.56,-0.53,0.29
1,0.38,0.38,-0.31,-0.09,1.52,1.35,1.49,3.81,2.33,1.34,...,0.31,1.28,1.40,0.00,-0.18,0.71,0.04,0.91,-0.79,0.22
2,-0.69,1.00,1.08,1.48,2.44,3.39,3.09,4.08,5.48,3.61,...,0.93,0.29,1.12,0.60,0.28,2.17,0.18,-0.09,-1.33,1.00
3,0.40,0.68,0.27,1.39,1.03,-0.32,-1.23,-0.50,0.11,0.87,...,1.18,0.43,-0.30,-0.07,-0.99,-0.75,1.11,1.35,-1.63,0.10
4,-0.81,1.59,-0.69,1.16,4.22,4.98,4.52,2.54,5.60,4.66,...,-0.07,0.62,0.14,1.33,-1.87,1.48,-0.02,-0.58,0.93,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.44,0.56,1.84,1.94,3.43,4.88,4.04,2.11,1.83,0.78,...,0.52,-2.56,-0.68,0.97,-0.78,-1.14,0.45,0.18,1.44,1.11
4996,1.18,-0.48,1.81,1.51,1.41,3.61,3.75,3.80,3.44,3.71,...,-0.41,0.37,-1.96,-1.19,-1.08,1.37,-1.02,0.71,-0.10,0.36
4997,0.64,0.81,-0.38,-0.88,1.55,0.10,0.42,-0.93,-0.70,1.54,...,0.49,1.61,-0.81,-1.35,-0.84,-0.64,1.19,-0.38,-0.70,-0.85
4998,0.18,1.65,1.91,2.07,4.28,3.61,4.46,4.62,4.80,0.25,...,0.17,-0.65,-0.73,0.89,-0.97,-1.30,-0.20,-0.63,-0.92,0.63


In [71]:
X_A, X_T, y_A , y_T = train_test_split(data, y, train_size=0.5, test_size=0.5)

# II. Simulation de l’aspect semi-supervisé

In [72]:
def remove_labels(labels: pd.DataFrame, taux: int = 50):
    index = np.array(labels.index)
    np.random.shuffle(index)
    nb_to_select = int((len(index)*taux)/100)
    index = index[:nb_to_select]
    for i in index:
        labels.loc[index] = -1
    return labels
y_A = remove_labels(y_A, 50)
y_A

2034   -1
1500   -1
2525   -1
2554   -1
4061   -1
       ..
1720    0
4108    0
837     2
2822    1
2262    0
Name: 40, Length: 2500, dtype: int64

# III. Sélection de variables semi-supervisée

In [93]:
np.mean(X_A[4].where(y_A == 2).dropna())

np.float64(1.337890818858561)

In [100]:
def s1(variable, y_A):
    sum_haut = 0
    sum_bas = 0
    for i in y_A.unique():
        n_i = len(y_A.where(y_A == i).dropna())
        u_i = np.mean(variable.where(y_A == i).dropna())
        u = np.mean(variable)
        print(np.mean(variable.where(y_A == i).dropna()))
        sum_haut += n_i*(u_i-u)**2
        #print(sum_haut)

        o_i = np.std(variable.where(y_A == i).dropna())
        sum_bas += n_i*o_i**2
    return sum_haut/sum_bas
for col in X_A.columns:
    s1(X_A[col], y_A)

0.6721280000000001
0.5794285714285715
0.7377667493796526
0.5959250585480094


np.float64(0.002085659090569174)

# IV. Evaluation de la sélection